# Load Model for Predictions & Evaluate Predictions

In [1]:
import sys
import pandas as pd
import numpy as np

In [2]:
sys.path.append("../../../python-package")
import lightgbm as lgb

## I. Read Model and Data

In [3]:
model = lgb.Booster(model_file='LightGBM_model.txt')

In [4]:
def parse_line(line):
    
    linesp = line.split()
    score = linesp[0]
    feats = linesp[1:]
    feats = {el.split(":")[0]:float(el.split(":")[1]) for el in feats}
    feats['score'] = float(score)
    
    return feats
    
def parse_test_file(filename, query_filename=None):
    
    with open(filename) as f:
        lines = f.readlines()
        
    test = pd.DataFrame.from_dict([parse_line(line) for line in lines])
    cols = test.columns.to_list()
    cols.remove("score")
    cols_sorted = sorted(cols, key=lambda txt: int(txt))
    
    test = test[['score'] + cols_sorted]
    
    if not query_filename: return test
    
    queries = pd.read_csv(query_filename, header=None).astype(int)
    #queries = queries.rename(columns={0:"qid"})
    test_q = test.copy()

    query_col = []
    # set arbitrary but unique query id
    for i, q in enumerate(queries[0]):
        query_col.extend([i]*q)
    len(query_col)

    test_q.insert(0, 'qid', query_col)
    
    return test_q
    
    
    

In [130]:
train = parse_test_file("../rank.train", "../rank.train.query")
train.head()

,qid,score,1,2,6,7,8,9,10,11,...,289,290,291,292,294,295,297,298,299,300
0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.89,0.75,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.43
1,1,1.0,0.69,NaN,NaN,NaN,NaN,NaN,NaN,0.64,...,NaN,0.53,NaN,NaN,NaN,NaN,0.36,NaN,NaN,0.25
2,1,0.0,0.69,NaN,NaN,NaN,NaN,NaN,NaN,0.64,...,NaN,0.75,NaN,NaN,NaN,NaN,0.36,NaN,NaN,0.43
3,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.64,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.43
4,1,0.0,0.69,NaN,NaN,0.72,NaN,NaN,NaN,0.64,...,NaN,0.59,NaN,NaN,NaN,NaN,0.36,NaN,NaN,0.33


In [131]:
def adjust_features_for_model(model, df, extra_cols=['qid', 'score']):
    
    feat_cols = [col for col in df.columns if col not in extra_cols]
    
    all_feats = pd.DataFrame(columns=model.feature_name())

    for col in feat_cols:
        col_name = f"Column_{col}"
        #print(col_name)
        all_feats[col_name] = df[col]
    all_feats = all_feats.astype(float)
        
    for col in extra_cols:
        all_feats.insert(0, col, df[col])
        
    return all_feats

In [132]:
train = adjust_features_for_model(model, train)

/Users/christian/opt/anaconda3/envs/searchrec/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3444: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  exec(code_obj, self.user_global_ns, self.user_ns)


In [133]:
train.head()

,score,qid,Column_0,Column_1,Column_2,Column_3,Column_4,Column_5,Column_6,Column_7,...,Column_291,Column_292,Column_293,Column_294,Column_295,Column_296,Column_297,Column_298,Column_299,Column_300
0,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.43
1,1.0,1,NaN,0.69,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.36,NaN,NaN,0.25
2,0.0,1,NaN,0.69,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.36,NaN,NaN,0.43
3,1.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.43
4,0.0,1,NaN,0.69,NaN,NaN,NaN,NaN,NaN,0.72,...,NaN,NaN,NaN,NaN,NaN,NaN,0.36,NaN,NaN,0.33


## II. Predictions

In [134]:
preds = model.predict(train[train.columns[2:]])

In [135]:
preds[:10]

array([ 0.01086442, -0.3786556 , -0.63524116, -0.64576398, -0.87648762,
       -0.13169932, -0.98025601,  0.22082931,  0.30044498, -0.89260241])

In [136]:
cli_preds = pd.read_csv("predict_result_train.txt", header=None)[0]

In [137]:
cli_preds[:10]

0    0.010864
1   -0.378656
2   -0.635241
3   -0.645764
4   -0.876488
5   -0.131699
6   -0.980256
7    0.220829
8    0.300445
9   -0.892602
Name: 0, dtype: float64

## III. Evaluate Predictions

In [138]:
train.insert(2, 'score_pred', preds)

In [139]:
train.head()

,score,qid,score_pred,Column_0,Column_1,Column_2,Column_3,Column_4,Column_5,Column_6,...,Column_291,Column_292,Column_293,Column_294,Column_295,Column_296,Column_297,Column_298,Column_299,Column_300
0,0.0,0,0.010864,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.43
1,1.0,1,-0.378656,NaN,0.69,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.36,NaN,NaN,0.25
2,0.0,1,-0.635241,NaN,0.69,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.36,NaN,NaN,0.43
3,1.0,1,-0.645764,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.43
4,0.0,1,-0.876488,NaN,0.69,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.36,NaN,NaN,0.33


In [140]:
train.isna().sum()

score            0
qid              0
score_pred       0
Column_0      3005
Column_1      1544
              ... 
Column_296    3005
Column_297     988
Column_298    2970
Column_299    2652
Column_300     195
Length: 304, dtype: int64

### Calculate NDCG@5 Score and confirm with Modeling Metrics

In [141]:
from sklearn.metrics import ndcg_score

In [142]:
def calc_ndcg_per_query(grp, k):
    
    if grp['score'].nunique()>1:
        return ndcg_score([grp['score'].values], [grp['score_pred'].values], k=k)
    else:
        # if query has only one doc
        return np.nan

In [143]:
ndcg_per_query = train.groupby("qid").apply(calc_ndcg_per_query, k=5)

In [144]:
ndcg_per_query.mean()

0.9852633534576977

In [145]:
train['score_pred'] = cli_preds

In [146]:
train.groupby("qid").apply(calc_ndcg_per_query, k=5).mean()

0.9852633534576977

> Prediction performance in agreement